##**Mchezopesa Ltd  football analysis.**

##**Defining the Question.**

###a) Specifying the Question

Prediction of match results based on which team is at Home and which is away, the type of the tourament ie Friendly, Fifa World Cup etc and the Rank of the team Approaches to be used

Approach 1: Polynomial approach
What to train given the :
Rank of home team
Rank of away team
tournament Type
Model 1: Predict how many goals the home team scores

Model 2: Predict how many goals the away team scores

Approach 2 Logistic approach

Feature Engineering: Figure out from the home team’s perspective if the game is a Win, Lose or Draw (W, L, D)

###b) Defining the Metric for Success

Correctly merge the two datasets
Perform Polynomial Regression and compare to the RMSE score
Archieve Logistic Regression

###c) Understanding the context

###d) Recording the Experimental Design


(CRISP- DM)Get the data, understand the data, Clean the data, Explore the data, Train the model, Test the model, Evaluate performance of the Model


##**Data Relevance**

###Loading the Libraries

In [0]:
#import pandas libraries
import pandas as pd
import pandas_profiling as pp
#Import numpy libraries
import numpy as np

#import matplot libraries
import matplotlib.pyplot as plt
%matplotlib inline

#import seaborn libraries
import seaborn as sns


# Import sklearn linear_model libraries 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error,  r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score 

import warnings
warnings.filterwarnings('ignore')




In [2]:
#Load results  dataset
# preview the first 5 row in a our dataset
results = pd.read_csv('results.csv')
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [3]:
#load ranking dataset
# preview the first 5 row in a our dataset
ranking = pd.read_csv('ranking.csv')
ranking.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


##**Access information about our datasets**

In [4]:
# Check the datatypes
ranking.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 16 columns):
rank                       57793 non-null int64
country_full               57793 non-null object
country_abrv               57793 non-null object
total_points               57793 non-null float64
previous_points            57793 non-null int64
rank_change                57793 non-null int64
cur_year_avg               57793 non-null float64
cur_year_avg_weighted      57793 non-null float64
last_year_avg              57793 non-null float64
last_year_avg_weighted     57793 non-null float64
two_year_ago_avg           57793 non-null float64
two_year_ago_weighted      57793 non-null float64
three_year_ago_avg         57793 non-null float64
three_year_ago_weighted    57793 non-null float64
confederation              57793 non-null object
rank_date                  57793 non-null object
dtypes: float64(9), int64(3), object(4)
memory usage: 7.1+ MB


Rank_date column has the wrong date type . It has to be converted to datetime type

In [0]:
ranking['rank_date'] =  pd.to_datetime(ranking['rank_date'], infer_datetime_format=True)

In [0]:
#Confirming rank_date has been converted
ranking.dtypes


rank                                int64
country_full                       object
country_abrv                       object
total_points                      float64
previous_points                     int64
rank_change                         int64
cur_year_avg                      float64
cur_year_avg_weighted             float64
last_year_avg                     float64
last_year_avg_weighted            float64
two_year_ago_avg                  float64
two_year_ago_weighted             float64
three_year_ago_avg                float64
three_year_ago_weighted           float64
confederation                      object
rank_date                  datetime64[ns]
dtype: object

In [0]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40839 entries, 0 to 40838
Data columns (total 9 columns):
date          40839 non-null object
home_team     40839 non-null object
away_team     40839 non-null object
home_score    40839 non-null int64
away_score    40839 non-null int64
tournament    40839 non-null object
city          40839 non-null object
country       40839 non-null object
neutral       40839 non-null bool
dtypes: bool(1), int64(2), object(6)
memory usage: 2.5+ MB


date column has the wrong date type . It has to be converted to datetime type

In [0]:
results['date'] =  pd.to_datetime(results['date'], infer_datetime_format=True)

In [0]:
#Confirming date column has been converted
results.dtypes

date          datetime64[ns]
home_team             object
away_team             object
home_score             int64
away_score             int64
tournament            object
city                  object
country               object
neutral                 bool
dtype: object

In [0]:
#Show the shape of the datasets
print(results.shape)
print(ranking.shape)

(40839, 9)
(57793, 16)


In [0]:
# We split the datetime column into month and year for both assistants
results['year'], results['month'],results['day'] = results['date'].dt.year, results['date'].dt.month, results['date'].dt.day
ranking['year'], ranking['month'], ranking['day'] = ranking['rank_date'].dt.year, ranking['rank_date'].dt.month,ranking['rank_date'].dt.day
# df['day'] = df['datetime_utc'].dt.day
# df['month'] = df['datetime_utc'].dt.month
# df['year'] = df['datetime_utc'].dt.year

In [0]:

print(results.shape)
print(ranking.shape)
results.head()

(40839, 12)
(57793, 19)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,1872,11,30
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,1873,3,8
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1874,3,7
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,1875,3,6
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,1876,3,4


In [0]:
#Now, we merge the two datasets
merged_fifa=results.merge( ranking,left_on=['country','year','month'], right_on=['country_full','year','month'],how='inner')
merged_fifa.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,month,day_x,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date,day_y
0,1993-08-01,Colombia,Paraguay,0,0,FIFA World Cup qualification,Barranquilla,Colombia,False,1993,8,1,19,Colombia,COL,0.0,36,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08,8
1,1993-08-15,Colombia,Argentina,2,1,FIFA World Cup qualification,Barranquilla,Colombia,False,1993,8,15,19,Colombia,COL,0.0,36,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08,8
2,1993-08-29,Colombia,Peru,4,0,FIFA World Cup qualification,Barranquilla,Colombia,False,1993,8,29,19,Colombia,COL,0.0,36,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08,8
3,1993-08-01,Peru,Argentina,0,1,FIFA World Cup qualification,Lima,Peru,False,1993,8,1,70,Peru,PER,0.0,16,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08,8
4,1993-08-08,Peru,Colombia,0,1,FIFA World Cup qualification,Lima,Peru,False,1993,8,8,70,Peru,PER,0.0,16,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08,8


In [0]:
merged_fifa.shape

(18808, 29)

###Creating a new dataframe with the column that are important to slve the problem stated above.

In [0]:
# creating a new data frame for ranking called rank_df whcich has variables that i think might be important for this analysis
final_data = merged_fifa[['rank','country_full','total_points','previous_points','rank_date','date', 'home_team', 'away_team', 'home_score', 'away_score','tournament', 'country', 'neutral']]
print(final_data.shape)
final_data.head()



(18808, 13)


,rank,country_full,total_points,previous_points,rank_date,date,home_team,away_team,home_score,away_score,tournament,country,neutral
0,19,Colombia,0.0,36,1993-08-08,1993-08-01,Colombia,Paraguay,0,0,FIFA World Cup qualification,Colombia,False
1,19,Colombia,0.0,36,1993-08-08,1993-08-15,Colombia,Argentina,2,1,FIFA World Cup qualification,Colombia,False
2,19,Colombia,0.0,36,1993-08-08,1993-08-29,Colombia,Peru,4,0,FIFA World Cup qualification,Colombia,False
3,70,Peru,0.0,16,1993-08-08,1993-08-01,Peru,Argentina,0,1,FIFA World Cup qualification,Peru,False
4,70,Peru,0.0,16,1993-08-08,1993-08-08,Peru,Colombia,0,1,FIFA World Cup qualification,Peru,False


###**Previewing our final dataset**

This will be done through pandas profiling

In [0]:
report = pp.ProfileReport(final_data)
report

Number of variables,13
Number of observations,18808
Total Missing (%),0.0%
Total size in memory,1.9 MiB
Average record size in memory,105.0 B
Numeric,5
Categorical,5
Boolean,1
Date,2
Text (Unique),0
Rejected,0



Dataset has 13 duplicate rows. We need to drop these.

In [0]:

#Dataset has 13 duplicate rows. We need to drop these.
final_data.drop_duplicates(keep=False,inplace=True) 
final_data.duplicated().sum()

0

In [0]:
final_data.isnull().sum()

rank               0
country_full       0
total_points       0
previous_points    0
rank_date          0
date               0
home_team          0
away_team          0
home_score         0
away_score         0
tournament         0
country            0
neutral            0
dtype: int64

our data has no missing values

In [0]:
final_data.columns

Index(['rank', 'country_full', 'total_points', 'previous_points', 'rank_date',
       'date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'country', 'neutral'],
      dtype='object')

###Checking for anomalies